In [1]:
import pickle
import numpy as np
import pandas as pd
import re

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [3]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    ffn_hyperparameter_search,
)

Reddit MoC

In [4]:
%run notebooks/Reddit_MoC/load_redditmoc.py

In [6]:
%run notebooks/Reddit_MoC/load_sbert-embeddings.py

In [7]:
sbert_embeddings.shape

torch.Size([6195, 384])

Baseline FFN

In [8]:
num_epochs = 100
batch = 32
hidden_dim_sizes = [[32, 32]]  # [[32,32], [64,64], [128,128], [256,256]]
dropout_rates = [0.2]  # [0.5, 0.2, 0.1]
learning_rates = [1e-4]  # [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"

In [9]:
ffn_current, best_ffn_current, _, __ = ffn_hyperparameter_search(
    num_epochs=num_epochs,
    x_data=sbert_embeddings,
    y_data=y_data,
    hidden_dim_sizes=hidden_dim_sizes,
    output_dim=output_dim,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    batch_size=batch,
    data_split_seed=123,
    split_ids=None,  # torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices=(
        df[df["set"] == "train"].index,
        df[df["set"] == "dev"].index,
        df[df["set"] == "test"].index,
    ),
    k_fold=False,
    validation_metric=validation_metric,
    results_output=None,
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
ffn_current

loss  accuracy        f1  \
0  0.591788  0.797529  0.487235   
0  0.568935  0.781369  0.480507   
0  0.582257  0.792776  0.482992   
0  0.590120  0.795627  0.485858   
0  0.598207  0.800380  0.489316   

                                       f1_scores  precision  \
0  [0.8965944272445822, 0.5651105651105651, 0.0]   0.475553   
0  [0.8888888888888888, 0.5526315789473685, 0.0]   0.457912   
0  [0.8923267326732675, 0.5566502463054188, 0.0]   0.471656   
0  [0.8952262864228147, 0.5623471882640587, 0.0]   0.473518   
0  [0.8986402966625463, 0.5693069306930694, 0.0]   0.478678   

                                precision_scores    recall  \
0  [0.8487690504103166, 0.5778894472361809, 0.0]  0.501005   
0  [0.8656716417910447, 0.5080645161290323, 0.0]  0.506385   
0  [0.8442622950819673, 0.5707070707070707, 0.0]  0.496488   
0   [0.8484136310223267, 0.572139303482587, 0.0]  0.500130   
0  [0.8492990654205608, 0.5867346938775511, 0.0]  0.502318   

                                   recall_scores  valid_loss  valid_accuracy  \
0  [0.9501312335958005, 0.5528846153846154, 0.0]    0.423041        0.868735   
0  [0.9133858267716536, 0.6057692307692307, 0.0]    0.393231        0.863962   
0  [0.9461942257217848, 0.5432692307692307, 0.0]    0.425457        0.881464   
0    [0.94750656167979, 0.5528846153846154, 0.0]    0.415761        0.874304   
0  [0.9540682414698163, 0.5528846153846154, 0.0]    0.434616        0.874304   

   ...  hidden_dim dropout_rate  learning_rate  seed  loss_function gamma  \
0  ...    (32, 32)          0.2         0.0001     0          focal     2   
0  ...    (32, 32)          0.2         0.0001     1          focal     2   
0  ...    (32, 32)          0.2         0.0001    12          focal     2   
0  ...    (32, 32)          0.2         0.0001   123          focal     2   
0  ...    (32, 32)          0.2         0.0001  1234          focal     2   

  k_fold  n_splits  batch_size  model_id  
0  False      None          32         0  
0  False      None          32         0  
0  False      None          32         0  
0  False      None          32         0  
0  False      None          32         0  

[5 rows x 26 columns]

In [11]:
best_ffn_current

loss  accuracy        f1  \
0  0.591788  0.797529  0.487235   
0  0.568935  0.781369  0.480507   
0  0.582257  0.792776  0.482992   
0  0.590120  0.795627  0.485858   
0  0.598207  0.800380  0.489316   

                                       f1_scores  precision  \
0  [0.8965944272445822, 0.5651105651105651, 0.0]   0.475553   
0  [0.8888888888888888, 0.5526315789473685, 0.0]   0.457912   
0  [0.8923267326732675, 0.5566502463054188, 0.0]   0.471656   
0  [0.8952262864228147, 0.5623471882640587, 0.0]   0.473518   
0  [0.8986402966625463, 0.5693069306930694, 0.0]   0.478678   

                                precision_scores    recall  \
0  [0.8487690504103166, 0.5778894472361809, 0.0]  0.501005   
0  [0.8656716417910447, 0.5080645161290323, 0.0]  0.506385   
0  [0.8442622950819673, 0.5707070707070707, 0.0]  0.496488   
0   [0.8484136310223267, 0.572139303482587, 0.0]  0.500130   
0  [0.8492990654205608, 0.5867346938775511, 0.0]  0.502318   

                                   recall_scores  valid_loss  valid_accuracy  \
0  [0.9501312335958005, 0.5528846153846154, 0.0]    0.423041        0.868735   
0  [0.9133858267716536, 0.6057692307692307, 0.0]    0.393231        0.863962   
0  [0.9461942257217848, 0.5432692307692307, 0.0]    0.425457        0.881464   
0    [0.94750656167979, 0.5528846153846154, 0.0]    0.415761        0.874304   
0  [0.9540682414698163, 0.5528846153846154, 0.0]    0.434616        0.874304   

   ...                            valid_recall_scores hidden_dim  \
0  ...  [0.9517958412098299, 0.6692913385826772, 0.0]   (32, 32)   
0  ...  [0.9366729678638941, 0.7480314960629921, 0.0]   (32, 32)   
0  ...  [0.9650283553875236, 0.6850393700787402, 0.0]   (32, 32)   
0  ...  [0.9593572778827977, 0.6614173228346457, 0.0]   (32, 32)   
0  ...  [0.9603024574669187, 0.6535433070866141, 0.0]   (32, 32)   

   dropout_rate learning_rate  seed loss_function gamma  k_fold  n_splits  \
0           0.2        0.0001     0         focal     2   False      None   
0           0.2        0.0001     1         focal     2   False      None   
0           0.2        0.0001    12         focal     2   False      None   
0           0.2        0.0001   123         focal     2   False      None   
0           0.2        0.0001  1234         focal     2   False      None   

   batch_size  
0          32  
0          32  
0          32  
0          32  
0          32  

[5 rows x 25 columns]

In [12]:
best_ffn_current["f1"].mean()

0.4851815427475053

In [13]:
np.stack(best_ffn_current["f1_scores"]).mean(axis=0)

array([0.89433533, 0.5612093 , 0.        ])